### 4-Recenter.ipynb

Recenters wings around the segmentated wing cells, removing variation in amount of visible wing base.

In [2]:
import cv2
import os
import shutil
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [3]:
# Define directories
data_dir = Path("/mnt/c/Projects/Master/Data/")
img_dir = data_dir / "Processed" / "AgePrediction" / "05-Recentered" / "1-LiveBeeWings"
mask_dir = data_dir / "Processed" / "AgePrediction" / "05-Recentered" / "2-Masks"
output_dir = data_dir / "Processed" / "AgePrediction" / "05-Recentered" / "3-Recentered"

DEBUG = False

In [5]:
# Make sure output dir exists
os.makedirs(output_dir, exist_ok=True)

# Image margin
MARGIN = 20  

for filename in tqdm(os.listdir(img_dir), desc="Processing files", ncols=145):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(img_dir, filename)
        mask_path = os.path.join(mask_dir, filename)

        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Create binary mask for labels 1–6
        combined_mask = ((mask > 0) & (mask <= 6)).astype(np.uint8)

        # Optional dilation to cover borders
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
        combined_mask = cv2.dilate(combined_mask, kernel, iterations=1)

        # Skip if mask is empty
        if np.count_nonzero(combined_mask) == 0:
            print(f"No labeled regions in {filename}")
            continue

        # Find bounding box of labeled region
        ys, xs = np.where(combined_mask > 0)
        y_min = max(np.min(ys) - MARGIN, 0)
        y_max = min(np.max(ys) + MARGIN, img.shape[0])
        x_min = max(np.min(xs) - MARGIN, 0)
        x_max = min(np.max(xs) + MARGIN, img.shape[1])

        # Crop the original image
        cropped_img = img[y_min:y_max, x_min:x_max]

        # Save cropped image
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, cropped_img)

Processing files: 100%|████████████████████████████████████████████████████████████████████████████████████████| 999/999 [01:23<00:00, 11.99it/s]
